## Joining on Non-Equi Operators

In [1]:
import pandas as pd
import janitor
import numpy as np

/opt/homebrew/Caskroom/miniforge/base/envs/pyjanitor-dev/lib/python3.9/site-packages/pandas_flavor/register.py:31: UserWarning: registration of accessor <class 'pandas_flavor.register.register_dataframe_method.<locals>.inner.<locals>.AccessorMethod'> under name 'then' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  register_dataframe_accessor(method.__name__)(AccessorMethod)


In [2]:
import numba
numba.__version__
import sys
sys.version

'3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:38:11) \n[Clang 14.0.6 ]'

In [3]:
from io import StringIO

data = """A    B   C      D          E
                       0  1  0.0  r  False   1970-01-01 00:00:00.000000001
                       1  0  0.0  r  False   1970-01-01 00:00:00.000000000"""

df = pd.read_csv(StringIO(data), sep=r'\s{2,}', engine='python', parse_dates=['E'])
df

,A,B,C,D,E
0,1,0.0,r,False,1970-01-01 00:00:00.000000001
1,0,0.0,r,False,1970-01-01 00:00:00.000000000


In [4]:
df.dtypes

A             int64
B           float64
C            object
D              bool
E    datetime64[ns]
dtype: object

In [5]:
data = """Integers  Numeric  Floats   Strings  Booleans      Dates    Dates_Right
                      0         0      0.0     0.0      r       False   1970-01-01  1970-01-01
                      1         1      0.0     0.0      r       False   1970-01-01  1970-01-01"""

right = pd.read_csv(StringIO(data), sep=r'\s{2,}', engine='python', parse_dates=['Dates','Dates_Right'])
right

,Integers,Numeric,Floats,Strings,Booleans,Dates,Dates_Right
0,0,0.0,0.0,r,False,1970-01-01,1970-01-01
1,1,0.0,0.0,r,False,1970-01-01,1970-01-01


In [6]:
right.dtypes


Integers                int64
Numeric               float64
Floats                float64
Strings                object
Booleans                 bool
Dates          datetime64[ns]
Dates_Right    datetime64[ns]
dtype: object

In [7]:
(
        df[["A", "E"]]
        .conditional_join(
            right[["Integers", "Dates"]],
            ("A", "Integers", "<="),
            ("E", "Dates", ">"),
            how="inner",
            use_numba=True,

            sort_by_appearance=False,
        )
)

,A,E,Integers,Dates
0,1,1970-01-01 00:00:00.000000001,1,1970-01-01


In [8]:
(
        df[["A", "E"]]
        .conditional_join(
            right[["Integers", "Dates"]],
            ("A", "Integers", "<="),
            ("E", "Dates", ">"),
            how="inner",
            use_numba=False,

            sort_by_appearance=False,
        )
)

,A,E,Integers,Dates
0,1,1970-01-01 00:00:00.000000001,1,1970-01-01


In [9]:
url = 'https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/results.csv'
events = pd.read_csv(url, parse_dates=['start', 'end']).iloc[:, 1:]
events.head()

,id,name,audience,start,sponsor,end
0,1,Event 1,1178,2022-11-19 10:00:00,Sponsor 2,2022-11-19 10:15:00
1,2,Event 2,1446,2015-09-27 15:00:00,Sponsor 11,2015-09-27 15:11:00
2,3,Event 3,2261,2019-11-12 18:00:00,Sponsor 10,2019-11-12 18:53:00
3,4,Event 4,1471,2019-12-24 22:00:00,Sponsor 6,2019-12-24 22:11:00
4,5,Event 5,2605,2028-06-20 12:00:00,Sponsor 8,2028-06-20 12:31:00


In [10]:
A = (events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '!='),
    use_numba = True,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)
A



left                                          right  \
         id               start                 end     id   
0      2345 1993-11-27 10:00:00 1993-11-27 12:00:00     10   
1     11178 1993-04-04 17:00:00 1993-04-04 17:22:00     15   
2     19605 2030-10-25 06:00:00 2030-10-25 08:00:00     17   
3      8218 2005-10-04 17:00:00 2005-10-04 17:27:00     26   
4     27696 2024-05-02 15:00:00 2024-05-02 15:07:00     35   
...     ...                 ...                 ...    ...   
3697  29375 2000-08-26 13:00:00 2000-08-26 13:53:00  29966   
3698  24173 2018-05-18 04:00:00 2018-05-18 04:36:00  29971   
3699    981 1992-06-07 22:00:00 1992-06-07 22:30:00  29978   
3700  19051 2025-06-05 01:00:00 2025-06-05 03:00:00  29984   
3701  12296 2016-09-04 14:00:00 2016-09-04 14:50:00  29995   

                                              
                   start                 end  
0    1993-11-27 12:00:00 1993-11-27 12:37:00  
1    1993-04-04 16:00:00 1993-04-04 18:00:00  
2    2030-10-25 07:00:00 2030-10-25 07:27:00  
3    2005-10-04 17:00:00 2005-10-04 17:18:00  
4    2024-05-02 15:00:00 2024-05-02 15:35:00  
...                  ...                 ...  
3697 2000-08-26 11:00:00 2000-08-26 13:00:00  
3698 2018-05-18 04:00:00 2018-05-18 04:18:00  
3699 1992-06-07 22:00:00 1992-06-07 22:23:00  
3700 2025-06-05 03:00:00 2025-06-05 03:17:00  
3701 2016-09-04 14:00:00 2016-09-04 14:32:00  

[3702 rows x 6 columns]

In [11]:
B = (events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '!='),
    use_numba = False,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)
B


left                                          right  \
         id               start                 end     id   
0        10 1993-11-27 12:00:00 1993-11-27 12:37:00   2345   
1        15 1993-04-04 16:00:00 1993-04-04 18:00:00  11178   
2        17 2030-10-25 07:00:00 2030-10-25 07:27:00  19605   
3        26 2005-10-04 17:00:00 2005-10-04 17:18:00   8218   
4        35 2024-05-02 15:00:00 2024-05-02 15:35:00   6916   
...     ...                 ...                 ...    ...   
3697  29966 2000-08-26 11:00:00 2000-08-26 13:00:00  29375   
3698  29971 2018-05-18 04:00:00 2018-05-18 04:18:00  24173   
3699  29978 1992-06-07 22:00:00 1992-06-07 22:23:00    981   
3700  29984 2025-06-05 03:00:00 2025-06-05 03:17:00  19051   
3701  29995 2016-09-04 14:00:00 2016-09-04 14:32:00  12296   

                                              
                   start                 end  
0    1993-11-27 10:00:00 1993-11-27 12:00:00  
1    1993-04-04 17:00:00 1993-04-04 17:22:00  
2    2030-10-25 06:00:00 2030-10-25 08:00:00  
3    2005-10-04 17:00:00 2005-10-04 17:27:00  
4    2024-05-02 15:00:00 2024-05-02 15:36:00  
...                  ...                 ...  
3697 2000-08-26 13:00:00 2000-08-26 13:53:00  
3698 2018-05-18 04:00:00 2018-05-18 04:36:00  
3699 1992-06-07 22:00:00 1992-06-07 22:30:00  
3700 2025-06-05 01:00:00 2025-06-05 03:00:00  
3701 2016-09-04 14:00:00 2016-09-04 14:50:00  

[3702 rows x 6 columns]

In [12]:
# cols = A.columns.tolist()
# A.sort_values(cols, ignore_index=True).equals(B.sort_values(cols, ignore_index=True))

In [13]:
c = {'vol': [35,15,5,35,18,90,17], 'profit':[45,35,55,12,15,55,11]}
d = {'vol':[20,50,15,16,40,20,40,2], 'profit':[30,10,12,52,35,20,30,57]}
c = pd.DataFrame(c)
d = pd.DataFrame(d)
l = c.conditional_join(d, ('vol','vol','<'),('profit','profit','>'), use_numba=True)
l

left        right       
    vol profit   vol profit
0    15     35    20     30
1     5     55    20     30
2    17     11    50     10
3    35     12    50     10
4    18     15    50     10
5    15     35    50     10
6    35     45    50     10
7     5     55    50     10
8     5     55    15     12
9     5     55    16     52
10   35     45    40     35
11    5     55    40     35
12   15     35    20     20
13    5     55    20     20
14   15     35    40     30
15   35     45    40     30
16    5     55    40     30

In [14]:
r = c.conditional_join(d, ('vol','vol','<'),('profit','profit','>'), use_numba=False)

In [15]:
# cols = l.columns.tolist()
# l.sort_values(cols, ignore_index=True).equals(r.sort_values(cols, ignore_index=True))

In [16]:
#https://stackoverflow.com/q/61948103/7175713 
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})

df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})

In [17]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [18]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


Join on equi and non-equi operators is possible:

In [19]:
df1.conditional_join(
        df2,
     #   ('id', 'id', '=='),
     ('value_1', 'value_2A', '>'),
     ('value_1', 'value_2B', '<'),
        
        
        sort_by_appearance = True,
        use_numba=True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2     3        1        3
1    1       5     2        3        6
2    2       3     2        2        4
3    3       4     1        3        5
4    3       4     2        3        6

The default join is inner. left and right joins are supported as well:

In [20]:
df1.conditional_join(
        df2,
     #   ('id', 'id', '=='),
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
    #    how='left',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2     3        1        3
1    1       5     2        3        6
2    2       3     2        2        4
3    3       4     1        3        5
4    3       4     2        3        6

In [21]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='right',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0  NaN     NaN     1        0        1
1  1.0     5.0     1        3        5
2  1.0     7.0     1        7        9
3  NaN     NaN     1       12       15
4  2.0     1.0     2        0        1
5  2.0     3.0     2        2        4
6  2.0     3.0     2        3        6
7  NaN     NaN     3        1        3

Join on just the non-equi joins is also possible:

In [22]:
df1.conditional_join(
        df2,
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
        how='inner',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2     3        1        3
1    1       5     2        3        6
2    2       3     2        2        4
3    3       4     1        3        5
4    3       4     2        3        6

Join on not equal -> !=

In [23]:
df1.conditional_join(
        df2,
        ('id', 'id', "!=")
    )

left         right                  
     id value_1    id value_2A value_2B
0     1       2     2        0        1
1     1       2     2        2        4
2     1       2     2        3        6
3     1       2     3        1        3
4     1       5     2        0        1
5     1       5     2        2        4
6     1       5     2        3        6
7     1       5     3        1        3
8     1       7     2        0        1
9     1       7     2        2        4
10    1       7     2        3        6
11    1       7     3        1        3
12    2       1     3        1        3
13    2       3     3        1        3
14    2       1     1        0        1
15    2       1     1        3        5
16    2       1     1        7        9
17    2       1     1       12       15
18    2       3     1        0        1
19    2       3     1        3        5
20    2       3     1        7        9
21    2       3     1       12       15
22    3       4     1        0        1
23    3       4     1        3        5
24    3       4     1        7        9
25    3       4     1       12       15
26    3       4     2        0        1
27    3       4     2        2        4
28    3       4     2        3        6

If the columns from both dataframes have nothing in common, a single indexed column is returned:

In [24]:
(df1.select_columns('value_1')
    .conditional_join(
        df2.select_columns('val*'),
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
    )
)

,value_1,value_2A,value_2B
0,2,1,3
1,5,3,6
2,3,2,4
3,4,3,5
4,4,3,6


Selection of relevant columns within `conditional_join`: 

In [25]:
df1.conditional_join(
        df2,
        ('id', 'id', "<"),
        df_columns = 'id',
        right_columns = 'id'
    )

,left,right
,id,id
0,1,2
1,1,2
2,1,2
3,1,3
4,1,2
5,1,2
6,1,2
7,1,3
8,1,2


Column renaming is also possible:

In [26]:
df1.conditional_join(
        df2,
        ('id', 'id', "<"),
        df_columns = {'id':'df_id'},
        right_columns = {'id':'right_id'}
    )

,df_id,right_id
0,1,2
1,1,2
2,1,2
3,1,3
4,1,2
5,1,2
6,1,2
7,1,3
8,1,2
9,1,2
